In [0]:
import pyspark.sql.functions as F

In [0]:
fato_envios_final = spark.table("grao.silver.fato_envios")
dim_localizacao = spark.table("grao.silver.dim_localizacao")
dim_metodo_envio = spark.table("grao.silver.dim_metodo_envio")
df_calendario = spark.table("grao.bronze.calendario") 

In [0]:
print("\n--- Criando Tabela Gold 2: Gold_Performance_por_Metodo_Envio ---")


gold_performance_metodo = (
    fato_envios_final
    .join(dim_metodo_envio, "id_metodo_envio", "left")
    .groupBy("metodo_de_envio")
    .agg(
        F.round((F.avg("chegou_no_tempo") * 100), 2).alias("taxa_entrega_no_prazo_percent"),
        F.round(F.avg("avaliacaoentrega"), 2).alias("avaliacao_media_da_entrega"),
        F.count("id_envio").alias("total_de_envios")
    )
    .orderBy(F.col("taxa_entrega_no_prazo_percent").desc())
)


gold_performance_metodo.write.mode("overwrite").format("delta").saveAsTable("grao.gold.performance_por_metodo_envio")

print("Tabela Gold 'performance_por_metodo_envio' criada com sucesso.")
